In [1]:
!nvidia-smi

Tue Aug 18 14:52:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce 920MX       Off  | 00000000:03:00.0 Off |                  N/A |
| N/A   41C    P0    N/A /  N/A |      5MiB /  2004MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# !git clone https://github.com/ShivamShrirao/dnn_from_scratch.git

In [3]:
# !git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# %cd deep_Q_learning_from_scratch

In [6]:
# !git pull

In [1]:
import gym
import time
import cv2
import numpy as np

from settings import *
from agent import *
from experience import *

In [2]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=1e-6, target_update_thresh=1000)
D_exp = ReplayMemory(capacity=1_000_000, nlap=2)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 128)        relu             73856
__________________________________________________________________________________________
3 Flatten(Flatten)        (None, 12800)              echo             0
__________________________________________________________________________________________
4 Den

In [3]:
env = gym.make('Pong-v0')

In [4]:
def preproc_obsv(obsv):
    obsv = cv2.cvtColor(obsv, cv2.COLOR_RGB2GRAY)
    obsv = obsv[34:194:2,::2]
    return obsv

In [5]:
D_exp.len

0

In [ ]:
for i_episode in range(1000):
    obinit = env.reset()
    if not i_episode:
        observation = obinit
    ep_score = 0
    start = time.time()
    for t in range(10_000):
#         env.render()
        action = agt.get_action(observation)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward

        D_exp.store_transition(preproc_obsv(observation), agt.actions.index(action), reward, done)
        observation = next_observation

        if (D_exp.len-D_exp.nxt_end) > BATCH_SIZE:
            agt.train(D_exp, BATCH_SIZE)
        if done:
            break
        print('\r', t, action, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon:.6f}, Time: {time.time()-start:.2f}")
    # agt.update_target()
    if not i_episode%4:
        agt.model.save_weights("/content/drive/My Drive/model.w8s")
env.close()

 949 3 -14.0  

In [ ]:
env.close()